**Faiss 是 Facebook AI Search 釋出的開源程式庫，不算是資料庫，概念是針對向量建立索引並進行高效率搜索。**

**HuggingFace的Dataset本身有支援Faiss(如下範例)**  
**Faiss 使用向量間的距離（通常是歐氏距離）來衡量相似度**

**Faiss的安裝,一般使用cpu版本就足夠**
```
pip install faiss-cpu
pip install faiss-gpu
```

### Step1 載入和準備數據集

In [ ]:
from datasets import load_dataset
dataset = load_dataset('roberthsu2003/data_for_RAG',split='train')
#增加一個columns,`text`,目的為組合question和answering內的文字,前綴加上`passage:`是RoBERTa類型的模形建議的
dataset = dataset.map(lambda x:{'text':
                      'passage:' + x['question'] + '\n' + x['answering']
                      })
dataset[:1]

{'question': ['如何查看 Model 3 最新的車主手冊資訊？'],
 'answering': ['您可以透過輕觸車輛觸控螢幕上的應用程式啟動工具，並選取「手冊」應用程式來查看最新且專屬於您車輛的車主手冊。[source: 309]'],
 'text': ['passage:如何查看 Model 3 最新的車主手冊資訊？\n您可以透過輕觸車輛觸控螢幕上的應用程式啟動工具，並選取「手冊」應用程式來查看最新且專屬於您車輛的車主手冊。[source: 309]']}

### Step2 載入分詞器和模型

In [3]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

In [7]:
#將modle,設定至cpu或gpu上執行
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.device

device(type='cpu')

### Step3 取出embedding(嵌入)

In [29]:
def get_embeddings(text_list):
    """text_list的型別是字串"""
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt", max_length=512
    ) #encoded_input的型別是BatchEncoding
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()} #將input_ids和attention_mask內的值(tensor),使用cpu或gpu做運算
    with torch.no_grad(): #使用model時不要做梯度運算
        model_output = model(**encoded_input) #BaseModelOutputWithPoolingAndCrossAttentions
        #model_output.last_hidden_state的Tensor,shape是([1,58,1024]),[batch_size, sequence_length, hidden_size]
        embeddings = model_output.last_hidden_state[:, 0, :]
        # [:, 0, :] 這個切片操作代表：
        # 第一個 : : 選取所有batch
        # 0 : 只取第一個token位置(即[CLS]標記)
        # 最後的 : : 選取所有hidden維度
        # [CLS]標記通常被用來表示整個句子的語義
        # 取出的維度變為[1,1024]
    return embeddings

In [39]:
#請參考-說明1和說明2

embedding_dataset = dataset.map(
    lambda x:{"embeddings":get_embeddings(x['text']).detach().cpu().numpy()[0]} #最後傳出得ndarray,shape是(1024,)
)


#測試用
# embedding_dataset = dataset.map(
#     lambda x:print(get_embeddings(x['text']).detach().cpu().numpy()[0])
# )


In [40]:
#使用FAISS進行高效的相似性搜索
#Dataset.add_faiss_index()
#說明3
embedding_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['question', 'answering', 'text', 'embeddings'],
    num_rows: 123
})

In [ ]:
#說明4
question = "充電埠在哪裡？"
question = "query:" + question
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 1024)

In [45]:
#scores是ndarray
#samples是dict
scores, samples = embedding_dataset.get_nearest_examples(
    "embeddings", question_embedding, k = 5
)


In [46]:
import pandas as pd
samples_df = pd.DataFrame.from_dict(samples)
samples_df['scores'] = scores
samples_df.sort_values('scores', ascending=True, inplace=True)

In [47]:
for _, row in samples_df.iterrows(): #請參考說明5
    print(f"question:",{row.question})
    print(f"answering:",{row.answering})
    print('score:',{row.scores})
    print("="*50)
    print()

question: {'Model 3 的充電埠在哪裡？'}
answering: {'充電埠位於車輛左後方，整合在尾燈組的一個蓋板後面。[source: 327]'}
score: {197.45962524414062}

question: {'如何使用 Model 3 的無線手機充電器？'}
answering: {'將支援 Qi 功能的手機放置在前中控台的兩個充電板之一即可充電。手機需直接接觸充電板，充電前請移除手機與充電器間的任何物體（金屬、NFC 卡等）。[source: 334]'}
score: {254.12985229492188}

question: {'Model 3 有多少個 USB 連接埠？分別在哪裡？'}
answering: {'Model 3 共有四個 USB 連接埠。一個 USB-C 在中控台後方置物空間，兩個 USB-C 在中控台後車廂出風口下方，一個 USB-A 在手套箱內。[source: 334]'}
score: {254.5594024658203}

question: {'哪個 USB 連接埠最適合用於儲存哨兵模式和行車記錄器影像？'}
answering: {'位於手套箱內的 USB-A 連接埠最適合儲存哨兵模式和行車記錄器影像，因為位置較安全。[source: 334]'}
score: {261.5010986328125}

question: {'Model 3 的低電壓電源插座在哪裡？其供電限制為何？'}
answering: {'低電壓電源插座位於中控置物台的置物空間中。該插座適用於需要持續電流高達 12A（峰值 16A）的配件。[source: 335]'}
score: {270.1339111328125}



# 說明1

### 1. 主要目的
這段程式碼是在將文字轉換成數值向量（embedding），並將這些向量儲存到dataset中的新欄位。

### 2. 程式碼解析


In [ ]:
embedding_dataset = dataset.map(
    lambda x:{"embeddings":get_embeddings(x['text']).detach().cpu().numpy()[0]}
)

- `dataset.map()`: 對dataset中的每一筆資料進行轉換
- `lambda x`: 匿名函數，`x`代表dataset中的每一筆資料
- `get_embeddings(x['text'])`: 
  - 將每筆資料中的'text'欄位轉換成向量
  - 回傳的是PyTorch tensor

### 3. 資料處理步驟
1. `.detach()`: 從計算圖中分離，不需要保留梯度
2. `.cpu()`: 將tensor移到CPU上
3. `.numpy()`: 將tensor轉換成numpy陣列
4. `[0]`: 取出第一個維度（因為get_embeddings返回的是batch形式）

### 4. 結果
- 在dataset中新增了一個名為"embeddings"的欄位
- 每一筆資料的"embeddings"欄位都包含了對應文字的向量表示
- 這些向量可以用來進行相似度比較

這樣處理後的資料，就可以用於後續的FAISS索引建立和相似度搜尋。

# 說明2

在 PyTorch 中，你可以通過以下幾種方式來檢查 Tensor 的相關資訊：

### 1. 檢查是否有梯度（Gradient）


In [ ]:
tensor = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

# 檢查是否需要梯度計算
print(tensor.requires_grad)  # True

# 檢查是否有梯度
print(tensor.grad is not None)  # False（還沒有進行反向傳播）



### 2. 檢查裝置（Device）位置


In [ ]:
# 檢查tensor在哪個裝置上
print(tensor.device)  # 會顯示 'cpu' 或 'cuda:0' 等

# 檢查是否在 GPU 上
print(tensor.is_cuda)  # True 或 False



### 3. 實際範例


In [ ]:
# 創建一個需要梯度的 tensor
x = torch.tensor([1.0, 2.0], requires_grad=True)
print(f"需要梯度計算: {x.requires_grad}")  # True
print(f"所在裝置: {x.device}")  # cpu

# 移到 GPU（如果有的話）
if torch.cuda.is_available():
    x = x.cuda()
    print(f"移動後的裝置: {x.device}")  # cuda:0

# 分離梯度
y = x.detach()
print(f"分離後是否需要梯度: {y.requires_grad}")  # False



### 4. 常見操作說明
- `tensor.requires_grad`: 檢查是否需要梯度計算
- `tensor.device`: 檢查tensor所在的裝置
- `tensor.is_cuda`: 檢查是否在 GPU 上
- `tensor.detach()`: 創建一個新的tensor，但不會有梯度計算
- `tensor.to(device)`: 將tensor移動到指定裝置

這些方法可以幫助你確認tensor的狀態和位置。

# 說明3

### 主要功能
這行程式碼是在 dataset 中建立 FAISS 索引，具體來說：

1. **建立索引**
- 對 dataset 中的 "embeddings" 欄位建立向量索引
- 這些 embeddings 是之前透過模型轉換得到的向量
- 索引的目的是加速後續的相似度搜尋

2. **技術細節**
- FAISS 會自動選擇適合的索引類型
- 預設使用 L2 距離（歐氏距離）來計算向量間的相似度
- 索引建立後會自動優化搜尋結構

### 實際應用


In [ ]:
# 建立索引後，可以使用 get_nearest_examples() 進行快速搜尋
scores, samples = embedding_dataset.get_nearest_examples(
    "embeddings",  # 使用哪個欄位的向量
    question_embedding,  # 搜尋的目標向量
    k = 5  # 要找出最相似的 5 筆資料
)



### 效能優勢
- 不需要逐一比較所有向量
- 透過索引結構快速找到相似向量
- 特別適合大規模資料集的相似度搜尋
- 可以處理高維度向量資料

這就是為什麼 FAISS 被廣泛應用在向量搜尋任務中的原因。

# 說明4

### 1. 步驟分解


In [ ]:
question = "充電埠在哪裡？"
question = "query:" + question  # 加上查詢前綴



接著進行向量轉換：


In [ ]:
question_embedding = get_embeddings([question]).cpu().detach().numpy()



### 2. 維度說明
- `get_embeddings([question])` 回傳的是一個 shape 為 `[1,1024]` 的 tensor 
  - 1: batch size (因為只有一個問題)
  - 1024: 向量維度
- 當使用 `.numpy()` 時，維持了原本的 2D 維度結構
- 這與之前的處理不同：
  ```python
  get_embeddings(x['text']).detach().cpu().numpy()[0]  # 之前的程式碼
  ```
  這裡的 `[0]` 是為了取出第一個維度，得到 1D array (1024,)

### 3. 為什麼保持 2D
- FAISS 的 `get_nearest_examples()` 需要 2D numpy array 作為輸入
- 這符合批次處理的標準格式
- shape (1,1024) 表示：
  - 1 個查詢
  - 每個查詢是 1024 維的向量

所以這裡不需要 `[0]`，因為我們需要保持矩陣的 2D 結構以符合 FAISS 的輸入要求。

# 說明5

在這個範例中，score 分數較低代表較好的答案，原因如下：

1. **距離計算原理**
- Faiss 使用向量間的距離（通常是歐氏距離）來衡量相似度
- 距離越小，表示兩個向量越接近，也就是內容越相似
- 在程式碼中的 `get_nearest_examples()` 方法回傳的 scores 即為這個距離值

2. **相似度與距離的關係**


In [ ]:
較相似 = 距離較小 = scores 較低
較不相似 = 距離較大 = scores 較高



3. **程式碼中的排序**


In [ ]:
samples_df.sort_values('scores', ascending=True, inplace=True)

- `ascending=True` 表示按照分數從小到大排序
- 這樣最相關的結果（距離最小的）會先顯示

4. **補充說明**
- 如果想要轉換成較直觀的相似度分數（越高越相似），可以：
  - 將距離轉換成相似度分數（例如使用餘弦相似度）
  - 或是將現有的距離分數做反向轉換（例如 1/distance 或 -distance）

這就是為什麼在結果中，較低的 score 反而代表較好的匹配結果。